# BFS/DFS Progression:

---

## 🟩 1. **Number of Islands (LC 200)** – baseline

* Learn: BFS/DFS, visited set, count connected components.
* Goal: Get used to looping over the grid, checking 4 directions, not revisiting cells.

---

## 🟨 2. **Max Area of Island (LC 695)**

* Same grid, same flood-fill.
* Instead of counting islands, compute the **size** of each island and return the maximum.
* Twist: carry an accumulator inside DFS/BFS.
* Builds: passing state (area count) through recursion/queue.

---

## 🟧 3. **Rotting Oranges (LC 994)**

* Grid flood-fill, but now BFS is **multi-source**.
* You enqueue all rotten oranges (`2`s) at once and spread rot to fresh oranges (`1`s) in waves.
* Learn: BFS in **layers** (minute by minute).
* Real use: simulates spread (disease, fire, network infection).

---

## 🟥 4. **Surrounded Regions (LC 130)**

* Grid of `'X'` and `'O'`.
* Any `'O'` connected to the **border** should survive; all others flip to `'X'`.
* Trick: Instead of counting islands, you protect border-connected regions.
* Builds: thinking **inverted** — mark what survives instead of what dies.

---

## 🟦 5. **Word Search (LC 79)**

* Grid of letters, search if a word exists.
* Use DFS, backtracking, visited markers.
* Twist: path-based rather than blob-based.
* Builds: understanding when to **backtrack** (undo visited marks after recursion).

---

## 🟪 6. **Pacific Atlantic Water Flow (LC 417)**

* Grid of heights. Water can flow downhill. Find cells that can reach **both oceans**.
* Twist: BFS/DFS **from the edges inward**, not from each cell outward.
* Builds: recognizing when to flip perspective (start from the goal instead of the source).

---

* **1 → 2:** same idea, different output.
* **2 → 3:** introduce BFS layers (time steps).
* **3 → 4:** shift from “count” to “transform.”
* **4 → 5:** add backtracking + path constraints.
* **5 → 6:** add graph directionality + reverse search.

---

# Number of Islands

Variants of Number of Islands pop up all over:

Computer Vision (CV): Connected Components

Imagine a binary image (black/white). Each “blob” of white pixels is an island. Counting islands = counting objects.

Use case: segmenting tumors in an MRI, finding letters in scanned text, counting cells in a microscope image.

Geospatial / Logistics

A city map with 1 = land, 0 = water. Islands are clusters of accessible land.

Or in USPS/transportation: each cluster of reachable delivery points without crossing obstacles is one “island.”

BFS is literally how routing algorithms discover contiguous service regions.

Network Security / Graph Analysis

Each node = computer, 1 = online, 0 = offline. BFS/DFS counts how many disconnected subnetworks (“islands”) exist.

Tells you how fragmented your system is after an outage.

Why interviewers love it

It tests if you can spot “graph hiding in a matrix.”

It forces you to handle boundaries and bookkeeping (visited).

It’s one step away from real problems (image segmentation, network partitioning

In [1]:
from typing import List, Tuple, Set
import collections  # added

class Solution:
    def islands(self, grid: List[List[str]]) -> int:
        if not grid:
            return 0
        
        rows, cols = len(grid), len(grid[0])
        visit = set()
        islands = 0

        def bfs(r, c):
            q = collections.deque()
            visit.add((r, c))
            q.append((r, c))

            while q:
                row, col = q.popleft()
                directions = [[1,0], [-1,0], [0,1], [0,-1]]  # fixed name

                for dr, dc in directions:  # added colon
                    r, c = row + dr, col + dc  # removed extra comma
                    if (r in range(rows) and
                        c in range(cols) and
                        grid[r][c] == "1" and
                        (r, c) not in visit):   # fixed parens/tuple
                        q.append((r, c))
                        visit.add((r, c))       # set.add, not append

        for r in range(rows):
            for c in range(cols):
                if grid[r][c] == "1" and (r, c) not in visit:  # removed stray '-'
                    bfs(r, c)
                    islands += 1

        return islands


In [2]:
from collections import deque           # queue for BFS (O(1) pops from left)
from typing import List, Tuple, Set      # type hints

def num_islands(grid: List[str]) -> int:
    """Count connected components of '1's in a grid given as list of strings."""
    if not grid or not grid[0]:          # empty grid or empty first row → 0 islands
        return 0

    rows, cols = len(grid), len(grid[0])    # dimensions
    visited: Set[Tuple[int, int]] = set()   # track explored cells so we don't repeat

    def bfs(sr: int, sc: int) -> None:   # BFS starting at (sr, sc)
        q = deque([(sr, sc)])            # init queue with the starting cell
        visited.add((sr, sc))            # mark start as visited
        while q:                         # process queue until empty
            r, c = q.popleft()           # pop next cell in the frontier
            # explore 4-directional neighbors (down, up, right, left)
            for nr, nc in ((r+1, c), (r-1, c), (r, c+1), (r, c-1)):
                if 0 <= nr < rows and 0 <= nc < cols:    # in bounds
                    if grid[nr][nc] == '1' and (nr, nc) not in visited:
                        visited.add((nr, nc))            # mark neighbor visited
                        q.append((nr, nc))               # enqueue for further expansion

    count = 0                               # total islands found
    for r in range(rows):                   # scan every cell row by row
        for c in range(cols):               # scan columns in the row
            if grid[r][c] == '1' and (r, c) not in visited:
                bfs(r, c)                   # eat the whole island via BFS
                count += 1                  # one more island discovered
    return count                            # return final tally


# ----- Example usage -----
grid = [
    "11010",
    "11000",
    "00101",
    "00111",
]

result = num_islands(grid)  # call the function to get the count
print(result)                    # -> 3


3


## Grid boilerplate

In [ ]:
# Directions (4-neighbor)
DIR4 = [(1,0), (-1,0), (0,1), (0,-1)]

def inb(r, c, R, C):
    return 0 <= r < R and 0 <= c < C

## DFS (recursive) – count/measure blobs (e.g., islands, max area)

In [1]:
from typing import List, Set, Tuple

def dfs_blob(grid: List[List[str]]) -> int:
    R, C = len(grid), len(grid[0])
    visited: Set[Tuple[int,int]] = set()

    def dfs(r: int, c: int) -> int:
        # assume caller checked grid[r][c] is land/'1'
        visited.add((r, c))
        size = 1  # flip to 0 if you're just marking, keep 1 if measuring area
        for dr, dc in DIR4:
            nr, nc = r + dr, c + dc
            if inb(nr, nc, R, C) and grid[nr][nc] == '1' and (nr, nc) not in visited:
                size += dfs(nr, nc)
        return size

    blobs = 0
    best = 0
    for r in range(R):
        for c in range(C):
            if grid[r][c] == '1' and (r, c) not in visited:
                area = dfs(r, c)
                blobs += 1
                best = max(best, area)
    return blobs  # or return best if problem asks "max area"


## DFS (in-place marking, O(1) extra space)

In [2]:
def dfs_mark(grid: List[List[str]]) -> int:
    R, C = len(grid), len(grid[0])

    def dfs(r: int, c: int):
        grid[r][c] = '0'  # mark visited
        for dr, dc in DIR4:
            nr, nc = r + dr, c + dc
            if inb(nr, nc, R, C) and grid[nr][nc] == '1':
                dfs(nr, nc)

    count = 0
    for r in range(R):
        for c in range(C):
            if grid[r][c] == '1':
                dfs(r, c)
                count += 1
    return count


## BFS (standard) – connected components without recursion

In [3]:
from collections import deque

def bfs_blob(grid: List[List[str]]) -> int:
    R, C = len(grid), len(grid[0])
    visited = set()

    def bfs(sr: int, sc: int) -> int:
        q = deque([(sr, sc)])
        visited.add((sr, sc))
        size = 0
        while q:
            r, c = q.popleft()
            size += 1
            for dr, dc in DIR4:
                nr, nc = r + dr, c + dc
                if inb(nr, nc, R, C) and grid[nr][nc] == '1' and (nr, nc) not in visited:
                    visited.add((nr, nc))
                    q.append((nr, nc))
        return size

    count = 0
    best = 0
    for r in range(R):
        for c in range(C):
            if grid[r][c] == '1' and (r, c) not in visited:
                area = bfs(r, c)
                count += 1
                best = max(best, area)
    return count  # or best


## BFS (level order) – multi-source + time steps (e.g., Rotting Oranges)

In [ ]:
from collections import deque

def bfs_levels(initial_sources, is_open, neighbors):
    """
    initial_sources: iterable of starting nodes
    is_open(u): predicate to accept/enqueue node u
    neighbors(u): iterable of neighbor nodes
    Returns minutes (levels) to exhaust spread; -1 if blocked (adjust per task).
    """
    q = deque()
    seen = set()

    # seed
    for s in initial_sources:
        if is_open(s):
            q.append(s)
            seen.add(s)

    minutes = -1
    while q:
        for _ in range(len(q)):
            u = q.popleft()
            for v in neighbors(u):
                if v not in seen and is_open(v):
                    seen.add(v)
                    q.append(v)
        minutes += 1
    return max(minutes, 0)


## Border-flood (e.g., Surrounded Regions, Pacific Atlantic)

Usage idea (Pacific Atlantic):

Build starts_pacific (top row + left col), starts_atlantic (bottom row + right col).

can_flow = heights[nr][nc] >= heights[r][c].

mark writes into a boolean visited grid and returns True on first touch.

In [4]:
from collections import deque

def flood_from_edges(starts, can_flow, mark):
    q = deque(starts)
    while q:
        r, c = q.popleft()
        if not mark(r, c):   # mark returns False if already marked/invalid
            continue
        for dr, dc in DIR4:
            nr, nc = r + dr, c + dc
            if can_flow(r, c, nr, nc):
                q.append((nr, nc))


## Backtracking template (e.g., Word Search)

In [ ]:
def exist(board: List[List[str]], word: str) -> bool:
    R, C = len(board), len(board[0])
    W = len(word)
    visited = [[False]*C for _ in range(R)]

    def dfs(r: int, c: int, i: int) -> bool:
        if i == W: 
            return True
        if not inb(r, c, R, C) or visited[r][c] or board[r][c] != word[i]:
            return False
        visited[r][c] = True
        for dr, dc in DIR4:
            if dfs(r + dr, c + dc, i + 1):
                return True
        visited[r][c] = False
        return False

    for r in range(R):
        for c in range(C):
            if dfs(r, c, 0):
                return True
    return False
